In [16]:
%load_ext autoreload
%autoreload 2

In [17]:
import os
os.chdir(os.path.expanduser("~/netml-project"))

In [66]:
import wandb

from helpers.pl_module import SeizurePredictor
from helpers.dataset import get_dataloaders, get_datasets

import torch
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer

from torchmetrics.functional.classification import binary_f1_score, binary_accuracy

import pandas as pd

### Setup

First things first, make sure that below you have your username if on izar, else, make sure that the CKPT_DIR is set to the correct path - where you want to download the checkpoint. In additin, also make sure that the SUBMISSION_DIR is set to the correct path - where you want to save the submission file. Ideally, you should save it in the root of the netml-project directory.

In [72]:
username = "cizinsky"
CKPT_DIR = f"/scratch/izar/{username}/netml/outputs/tmp"
SUBMISSION_DIR = f"/home/{username}/netml-project/submissions"

In [20]:
!mkdir -p $CKPT_DIR

### Download the best model from wandb and load it

First, filter the runs by tag for instance ang get an overview.

In [21]:
api = wandb.Api()

runs = api.runs("ludekcizinsky/seizure-prediction")

tagged_runs = [run for run in runs if "dev" in run.tags]

for run in tagged_runs:
    print(f"{run.id} | {run.name} | tags: {run.tags} | val/f1: {run.summary.get('val/f1')}")

trb0772b | rural-wildflower-56 | tags: ['dev'] | val/f1: 0.6857143640518188


Next, you can choose specific run (based on the run id - most left column) and download the checkpoint.

In [22]:
run_id = "trb0772b"
run = next((run for run in runs if run.id == "trb0772b"), None)
assert run is not None, "Run not found!"

artifact_path = f"ludekcizinsky/seizure-prediction/model-{run_id}:v2"
artifact = api.artifact(artifact_path, type="model")
artifact.download(CKPT_DIR)
print(f"Downloaded checkpoint to {CKPT_DIR}.")

wandb:   1 of 1 files downloaded.  


Downloaded checkpoint to /scratch/izar/cizinsky/netml/outputs/tmp.


Finally, load the model from the checkpoint and set it to evaluation mode.

In [23]:
pl_module = SeizurePredictor.load_from_checkpoint(f"{CKPT_DIR}/model.ckpt")
pl_module.eval().freeze()

### Inference

Finally, inference time! Let's start with loading the val data.

In [26]:
_, val_dataloader = get_dataloaders(pl_module.hparams)
trainer = Trainer(accelerator="gpu", devices=1, logger=False, callbacks=[])

FYI: using the following signal transform: normalize -> fft_filtering


/home/cizinsky/venvs/netml/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /home/cizinsky/venvs/netml/lib/python3.10/site-pack ...
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Next, we can predict the outputs, and map them into single tensor.

In [30]:
# Predict
outputs = trainer.predict(pl_module, val_dataloader)

# Map into single tensor
preds = torch.cat([output["preds_batch"] for output in outputs])
y = torch.cat([output["y_batch"] for output in outputs])

/home/cizinsky/venvs/netml/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /home/cizinsky/venvs/netml/lib/python3.10/site-pack ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Finally, we can compute the metrics.

In [41]:
acc = binary_accuracy(preds, y)
f1 = binary_f1_score(preds, y)
print(f"Accuracy: {acc:.4f}, F1: {f1:.4f}")

Accuracy: 0.8900, F1: 0.6857


### Test set inference

Finally, now that we have loaded the model, evaluated it on the val set, we can test it on the test set. Let's start loading the test set:

In [ ]:
test_dataset = get_datasets(pl_module.hparams, split="test")
test_dataloader = DataLoader(
    test_dataset,
    batch_size=pl_module.hparams.data.batch_size,
    num_workers=pl_module.hparams.data.num_workers,
    shuffle=False,
)

FYI: using the following signal transform: normalize -> fft_filtering


Next, we run the inference and collect the results:

In [67]:
outputs = trainer.predict(pl_module, test_dataloader)
preds = torch.cat([output["preds_batch"] for output in outputs]).cpu().numpy()
sample_ids = []
for output in outputs:
    sample_ids.extend(output["y_batch"])

/home/cizinsky/venvs/netml/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /home/cizinsky/venvs/netml/lib/python3.10/site-pack ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Finally, we create the submission file:

In [70]:
submission_df = pd.DataFrame({"id": sample_ids, "label": preds})
submission_df.head()

,id,label
0,pqejgcvm_s001_t000_0,0
1,pqejgcvm_s001_t000_1,0
2,pqejgcvm_s001_t000_2,1
3,pqejgcvm_s001_t000_3,0
4,pqejgcvm_s001_t000_4,0


In [73]:
subm_path = os.path.join(SUBMISSION_DIR, f"run_{run_id}.csv")
submission_df.to_csv(subm_path, index=False)
print(f"Kaggle submission file generated: {subm_path}")

Kaggle submission file generated: /home/cizinsky/netml-project/submissions/run_trb0772b.csv
